In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 500

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

In [6]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

In [7]:
# pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )

In [8]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )



In [9]:
len(pt_features[pt_features['isCase']==True])

15209

In [10]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_end,data_start,matchid,reason_for_removal,age_at_index_date,insomnia,stroke,heart_failure,sleep_apnoea,chronic_pulmonary_disease,epilepsy,other_sedatives_100_pdds,antipsychotics_100_pdds,antidepressants_100_pdds,non_insomnia_GP_consultations,insomnia_any,insomnia_count:0,insomnia_count:1_5,insomnia_count:above_5,non_insomnia_GP_consultations:0,non_insomnia_GP_consultations:1_10,non_insomnia_GP_consultations:11_100,non_insomnia_GP_consultations:101_1000,non_insomnia_GP_consultations:above_1000,age_at_index_date:65-69,age_at_index_date:70-74,age_at_index_date:75-79,age_at_index_date:80-84,age_at_index_date:85-89,age_at_index_date:90-99,age_at_index_date:above_99,antidepressant_pdds:00000,antidepressant_pdds:00001_10,antidepressant_pdds:00011_100,antidepressant_pdds:00101_1000,antidepressant_pdds:01001_10000,antidepressant_pdds:10000_and_above,antipsychotic_pdds:00000,antipsychotic_pdds:00001_10,antipsychotic_pdds:00011_100,antipsychotic_pdds:00101_1000,antipsychotic_pdds:01001_10000,antipsychotic_pdds:10000_and_above,other_sedative_pdds:00000,other_sedative_pdds:00001_10,other_sedative_pdds:00011_100,other_sedative_pdds:00101_1000,other_sedative_pdds:01001_10000,other_sedative_pdds:10000_and_above,mental_illness,mood_stabilisers_and_AEDs_100_pdds,lithium_100_pdds,z_drugs_100_pdds,benzodiazepines_100_pdds,benzodiazepine_pdds:00000,benzodiazepine_pdds:00001_10,benzodiazepine_pdds:00011_100,benzodiazepine_pdds:00101_1000,benzodiazepine_pdds:01001_10000,benzodiazepine_pdds:10000_and_above,z_drug_pdds:00000,z_drug_pdds:00001_10,z_drug_pdds:00011_100,z_drug_pdds:00101_1000,z_drug_pdds:01001_10000,z_drug_pdds:10000_and_above,mood_stabilisers_and_AED_pdds:00000,mood_stabilisers_and_AED_pdds:00001_10,mood_stabilisers_and_AED_pdds:00011_100,mood_stabilisers_and_AED_pdds:00101_1000,mood_stabilisers_and_AED_pdds:01001_10000,mood_stabilisers_and_AED_pdds:10000_and_above,lithium_pdds:00000,lithium_pdds:00001_10,lithium_pdds:00011_100,lithium_pdds:00101_1000,lithium_pdds:01001_10000,lithium_pdds:10000_and_above
19195,9550164,17,164,1,2007-10-10,True,NaN,2010-07-23,1995-01-31,24296,NaN,90,0,0,0,0,0,0,0.0,0.000000,0.000000,159,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.0,0.0,32.12831,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
15661,10945031,20,31,1,2003-07-24,False,NaN,2006-10-10,1992-01-31,36205,NaN,83,0,0,1,0,0,0,0.0,0.474043,0.506837,110,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,6.007793,0.0,0.0,0.00000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4381,27968120,23,120,1,2008-07-28,False,NaN,2010-11-24,1998-03-30,33141,NaN,85,0,0,0,0,0,0,0.0,0.000000,0.000000,107,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.0,0.0,0.00000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1834,101220,13,220,1,2007-02-09,False,NaN,2009-03-25,1997-01-31,29267,NaN,94,0,1,0,0,0,0,0.0,0.000000,0.000000,111,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.0,0.0,0.00000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8301,5717484,29,484,1,2010-10-11,False,NaN,2013-03-18,1999-11-04,33062,NaN,81,0,0,0,0,0,0,0.0,0.000000,0.000000,276,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.00000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [11]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_end', 'data_start', 'matchid',
       'reason_for_removal', 'age_at_index_date', 'insomnia', 'stroke',
       'heart_failure', 'sleep_apnoea', 'chronic_pulmonary_disease',
       'epilepsy', 'other_sedatives_100_pdds', 'antipsychotics_100_pdds',
       'antidepressants_100_pdds', 'non_insomnia_GP_consultations',
       'insomnia_any', 'insomnia_count:0', 'insomnia_count:1_5',
       'insomnia_count:above_5', 'non_insomnia_GP_consultations:0',
       'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:above_1000', 'age_at_index_date:65-69',
       'age_at_index_date:70-74', 'age_at_index_date:75-79',
       'age_at_index_date:80-84', 'age_at_index_date:85-89',
       'age_at_index_date:90-99', 'age_at_index_date:above_99',
       'antidepressant_pdds:00000', 'antid

In [12]:
characteristics = [column for column in pt_features.columns if column not in [
    'patid',  'yob', 'pracid', 'index_date', 'isCase',
    'final dementia medcode', 'data_start', 'data_end', 'matchid']]

In [13]:
baselines = add_baseline_characteristics(characteristics,pt_features)

In [14]:
len(pt_features[pt_features['isCase']==False]),len(pt_features[pt_features['isCase']==True])

(15209, 15209)

In [15]:
baselines[['Cases','Controls']]

,Cases,Controls
female,10055 (66.1%),10055 (66.1%)
stroke,1662 (10.9%),1120 (7.4%)
heart_failure,913 (6.0%),726 (4.8%)
sleep_apnoea,25 (0.2%),19 (0.1%)
chronic_pulmonary_disease,3449 (22.7%),2512 (16.5%)
epilepsy,288 (1.9%),165 (1.1%)
insomnia_any,1121 (7.4%),708 (4.7%)
insomnia_count:0,14088 (92.6%),14501 (95.3%)
insomnia_count:1_5,1065 (7.0%),687 (4.5%)
insomnia_count:above_5,56 (0.4%),21 (0.1%)
